# Credit Score

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.ensemble import RandomForestClassifier

## Load data

In [2]:
df_train = pd.read_csv("train_data.csv")
df_test = pd.read_csv("test_data.csv")
df_train.head()

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,0x6762,CUS_0x6fe2,January,Schnurra,25,164-17-6558,Accountant,62214.36,4997.530000,9,...,_,2892.91,31.737712,13 Years and 10 Months,Yes,339.971582,216.583807190171,High_spent_Small_value_payments,203.19761100877272,-1
1,0x18b64,CUS_0xb13c,July,Jennifert,33,528-06-6283,_______,14613.65,1334.804167,8,...,Bad,2882.07,27.414562,2 Years and 3 Months,Yes,67.975819,57.54453501725725,High_spent_Small_value_payments,267.96006258969163,0
2,0xb8d5,CUS_0xa0c3,August,Chanf,56,355-23-1301,Lawyer,101112.6,8600.050000,8,...,Good,399.49,36.516160,29 Years and 3 Months,No,143.856735,132.07082261639974,High_spent_Large_value_payments,824.0774428659337,1
3,0x2ec3,CUS_0x6732,February,alazsh,50,060-69-0738,Journalist,140390.32,11888.193333,5,...,Good,1423.23,36.955252,30 Years and 9 Months,No,182.160424,133.2130344228415,High_spent_Medium_value_payments,1123.4458751336915,1
4,0x1b36,CUS_0x8c69,January,Toonkelf,24,070-07-9414,Musician,20188.02,1851.335000,4,...,_,399.92,28.542585,20 Years and 2 Months,No,25.580281,147.63621347336863,Low_spent_Large_value_payments,281.91700571081503,0


## Subtask 1

In [3]:
num_train_rows = len(df_train)
num_train_rows

44207

In [4]:
subtask1_rows = [(1, 1, num_train_rows)]

## Subtask 2

In [5]:
filtered = df_train.loc[df_train["Credit_Utilization_Ratio"] >= 25]
mean_salary = int(filtered["Monthly_Inhand_Salary"].mean().item())
mean_salary

4106

In [6]:
subtask2_rows = [(2, 1, mean_salary)]

## Subtask 3

In [7]:
months = df_train["Month"].unique()
num_unique_months = len(months)
months, num_unique_months

(array(['January', 'July', 'August', 'February', 'May', 'March', 'June',
        'April'], dtype=object),
 8)

In [8]:
subtask3_rows = [(3, 1, num_unique_months)]

## Subtask 4

In [9]:
num_ssn_20 = len(df_train.loc[df_train["SSN"].str.endswith("20")]["SSN"].unique())
num_ssn_20

109

In [10]:
subtask4_rows = [(4, 1, num_ssn_20)]

## Subtask 5

In [11]:
df_train.dtypes

ID                           object
Customer_ID                  object
Month                        object
Name                         object
Age                          object
SSN                          object
Occupation                   object
Annual_Income                object
Monthly_Inhand_Salary       float64
Num_Bank_Accounts             int64
Num_Credit_Card               int64
Interest_Rate                 int64
Num_of_Loan                  object
Type_of_Loan                 object
Delay_from_due_date           int64
Num_of_Delayed_Payment       object
Changed_Credit_Limit         object
Num_Credit_Inquiries        float64
Credit_Mix                   object
Outstanding_Debt             object
Credit_Utilization_Ratio    float64
Credit_History_Age           object
Payment_of_Min_Amount        object
Total_EMI_per_month         float64
Amount_invested_monthly      object
Payment_Behaviour            object
Monthly_Balance              object
Credit_Score                

In [12]:
skipped_cols = ["ID", "Customer_ID", "Month", "Name", "SSN"]
num_cols = ["Age", "Annual_Income", "Monthly_Inhand_Salary", "Num_Bank_Accounts", "Num_Credit_Card", 
            "Interest_Rate", "Num_of_Loan", "Delay_from_due_date", "Num_of_Delayed_Payment", "Changed_Credit_Limit",
            "Num_Credit_Inquiries", "Outstanding_Debt", "Credit_Utilization_Ratio", "Credit_History_Age", "Total_EMI_per_month",
            "Amount_invested_monthly", "Monthly_Balance"]
cat_cols = ["Occupation", "Type_of_Loan", "Credit_Mix", "Payment_of_Min_Amount", "Payment_Behaviour"]
ans_col = "Credit_Score"
len(skipped_cols + num_cols + cat_cols) + 1 == len(df_train.columns)  # Make sure no miss

True

In [13]:
X_train, y_train = df_train[num_cols + cat_cols].copy(), df_train[ans_col].copy()
X_test = df_test[num_cols + cat_cols].copy()

### Handle categorical features

In [14]:
for col in cat_cols:
    print(X_train[col].value_counts(), end="\n\n")

Occupation
_______          3086
Lawyer           2971
Mechanic         2831
Engineer         2788
Doctor           2781
Developer        2779
Media_Manager    2775
Teacher          2775
Accountant       2743
Entrepreneur     2741
Scientist        2729
Architect        2724
Musician         2667
Writer           2635
Manager          2599
Journalist       2583
Name: count, dtype: int64

Type_of_Loan
Not Specified                                                                                                                      725
Personal Loan                                                                                                                      657
Credit-Builder Loan                                                                                                                652
Debt Consolidation Loan                                                                                                            636
Student Loan                                             

#### Handle 'Type_of_Loan' - multi-hot encoding

In [15]:
X_train["Type_of_Loan"] = X_train["Type_of_Loan"].str.replace(", and ", ", ").str.split(", ")

In [16]:
mlb = MultiLabelBinarizer()
multi_hot_train = pd.DataFrame(mlb.fit_transform(X_train["Type_of_Loan"]), columns=mlb.classes_, dtype=bool)
multi_hot_train

,Auto Loan,Credit-Builder Loan,Debt Consolidation Loan,Home Equity Loan,Mortgage Loan,Not Specified,Payday Loan,Personal Loan,Student Loan
0,False,True,False,False,True,False,True,False,True
1,False,True,True,False,True,False,False,True,False
2,True,False,False,False,True,False,False,False,False
3,False,True,False,True,False,False,True,False,False
4,False,False,False,False,True,False,False,False,True
...,...,...,...,...,...,...,...,...,...
44202,True,False,False,False,False,False,False,False,False
44203,False,False,True,True,False,False,False,False,False
44204,False,True,True,False,False,False,True,False,True
44205,True,True,False,False,True,False,True,True,False


In [17]:
X_train = pd.concat([X_train.drop("Type_of_Loan", axis=1), multi_hot_train], axis=1)

In [18]:
X_test["Type_of_Loan"] = X_test["Type_of_Loan"].str.replace(", and ", ", ").str.split(", ")
multi_hot_test = pd.DataFrame(mlb.transform(X_test["Type_of_Loan"]), columns=mlb.classes_)
X_test = pd.concat([X_test.drop("Type_of_Loan", axis=1), multi_hot_test], axis=1)

In [19]:
cat_cols.remove("Type_of_Loan")

#### Handle other categorical features - one-hot encoding

In [20]:
X_train = pd.get_dummies(X_train, columns=cat_cols)
X_test = pd.get_dummies(X_test, columns=cat_cols)
X_train.dtypes

Age                                                    object
Annual_Income                                          object
Monthly_Inhand_Salary                                 float64
Num_Bank_Accounts                                       int64
Num_Credit_Card                                         int64
Interest_Rate                                           int64
Num_of_Loan                                            object
Delay_from_due_date                                     int64
Num_of_Delayed_Payment                                 object
Changed_Credit_Limit                                   object
Num_Credit_Inquiries                                  float64
Outstanding_Debt                                       object
Credit_Utilization_Ratio                              float64
Credit_History_Age                                     object
Total_EMI_per_month                                   float64
Amount_invested_monthly                                object
Monthly_

### Handle numerical features

In [21]:
X_train[num_cols].dtypes

Age                          object
Annual_Income                object
Monthly_Inhand_Salary       float64
Num_Bank_Accounts             int64
Num_Credit_Card               int64
Interest_Rate                 int64
Num_of_Loan                  object
Delay_from_due_date           int64
Num_of_Delayed_Payment       object
Changed_Credit_Limit         object
Num_Credit_Inquiries        float64
Outstanding_Debt             object
Credit_Utilization_Ratio    float64
Credit_History_Age           object
Total_EMI_per_month         float64
Amount_invested_monthly      object
Monthly_Balance              object
dtype: object

In [22]:
obj_num_cols = list(filter(lambda x: X_train[x].dtypes == "object", num_cols))
obj_num_cols

['Age',
 'Annual_Income',
 'Num_of_Loan',
 'Num_of_Delayed_Payment',
 'Changed_Credit_Limit',
 'Outstanding_Debt',
 'Credit_History_Age',
 'Amount_invested_monthly',
 'Monthly_Balance']

In [23]:
for col in obj_num_cols:
    print(f"{col:<25}", ", ".join(sorted(set("".join(X_train[col])))))  # List characters used in each column

Age                       -, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, _
Annual_Income             ., 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, _
Num_of_Loan               -, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, _
Num_of_Delayed_Payment    -, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, _
Changed_Credit_Limit      -, ., 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, _
Outstanding_Debt          ., 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, _
Credit_History_Age         , 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, M, Y, a, d, e, h, n, o, r, s, t
Amount_invested_monthly   ., 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, _
Monthly_Balance           -, ., 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, _


#### Handle 'Credit_History_Age' - with specific format

In [24]:
np.all(X_train["Credit_History_Age"].str.contains(r"^\d+ Years and \d+ Months$")).item()  # Do all records follow this format?

True

In [25]:
def parse_credit_history_age(x):
    year, month = x.replace(" Months", "").split(" Years and ")
    return int(year) + int(month) / 12

In [26]:
X_train["Credit_History_Age"] = X_train["Credit_History_Age"].apply(parse_credit_history_age)
X_test["Credit_History_Age"] = X_test["Credit_History_Age"].apply(parse_credit_history_age)
X_train["Credit_History_Age"]

0        13.833333
1         2.250000
2        29.250000
3        30.750000
4        20.166667
           ...    
44202    29.583333
44203    19.833333
44204    12.000000
44205    10.916667
44206    31.750000
Name: Credit_History_Age, Length: 44207, dtype: float64

In [27]:
obj_num_cols.remove("Credit_History_Age")

#### Handle numerical features with dtype 'object'

In [28]:
for col in obj_num_cols:
    X_train[col] = X_train[col].str.replace("_", "").replace('', np.nan).astype(np.float64)
    X_test[col] = X_test[col].str.replace("_", "").replace('', np.nan).astype(np.float64)

In [29]:
X_train[obj_num_cols].isna().any(), X_test[obj_num_cols].isna().any()

(Age                        False
 Annual_Income              False
 Num_of_Loan                False
 Num_of_Delayed_Payment     False
 Changed_Credit_Limit        True
 Outstanding_Debt           False
 Amount_invested_monthly    False
 Monthly_Balance            False
 dtype: bool,
 Age                        False
 Annual_Income              False
 Num_of_Loan                False
 Num_of_Delayed_Payment     False
 Changed_Credit_Limit        True
 Outstanding_Debt           False
 Amount_invested_monthly    False
 Monthly_Balance            False
 dtype: bool)

In [30]:
X_train["Changed_Credit_Limit"] = X_train["Changed_Credit_Limit"].fillna(0)
X_test["Changed_Credit_Limit"] = X_test["Changed_Credit_Limit"].fillna(0)

#### Bound values

In [31]:
X_train[num_cols].describe()

,Age,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Total_EMI_per_month,Amount_invested_monthly,Monthly_Balance
count,44207.000000,4.420700e+04,44207.000000,44207.000000,44207.000000,44207.000000,44207.000000,44207.000000,44207.000000,44207.000000,44207.000000,44207.000000,44207.000000,44207.000000,44207.000000,44207.000000,4.420700e+04
mean,112.512928,1.757612e+05,4019.547959,16.771281,23.223109,77.300925,3.432036,22.054652,31.557152,10.514315,26.791504,1509.416484,32.213410,17.757895,1451.435871,625.587657,-3.770142e+22
std,698.366011,1.441371e+06,3092.839667,114.890173,130.872496,483.025792,62.831604,15.253947,227.905606,7.009926,185.571018,1189.364396,5.055008,8.343751,8446.131137,2046.340581,3.544856e+24
min,-500.000000,7.005930e+03,303.645417,-1.000000,0.000000,1.000000,-100.000000,-5.000000,-3.000000,-6.480000,0.000000,0.230000,21.264254,0.166667,4.462837,0.000000,-3.333333e+26
25%,24.000000,1.886328e+04,1574.695833,4.000000,4.000000,8.000000,2.000000,10.000000,9.000000,5.210000,3.000000,607.785000,28.035357,11.166667,41.132527,72.205544,2.645375e+02
50%,33.000000,3.597149e+04,2986.346667,6.000000,6.000000,15.000000,3.000000,19.000000,14.000000,9.550000,6.000000,1243.130000,32.239814,17.666667,78.369329,130.274410,3.248861e+02
75%,42.000000,6.984002e+04,5704.220000,8.000000,7.000000,22.000000,6.000000,29.000000,19.000000,15.320000,9.000000,2132.460000,36.402076,24.333333,168.955552,250.932553,4.433650e+02
max,8697.000000,2.419806e+07,15204.633333,1798.000000,1499.000000,5789.000000,1485.000000,67.000000,4397.000000,36.090000,2594.000000,4998.070000,49.564519,33.666667,82331.000000,10000.000000,1.552946e+03


In [32]:
def bound(col, lower, upper):
    for df in (X_train, X_test):
        df[col] = df[col].where((df[col] >= lower) & (df[col] <= upper), np.nan)

In [33]:
bound("Age", 18, 100)
bound("Num_Bank_Accounts", 0, 10)
bound("Num_Credit_Card", 0, 10)
bound("Interest_Rate", 0, 100)
bound("Num_of_Loan", 0, 50)
bound("Delay_from_due_date", 0, np.inf)
bound("Num_of_Delayed_Payment", 0, 50)
bound("Num_Credit_Inquiries", 0, 30)
bound("Monthly_Balance", 0, np.inf)

In [34]:
X_train

,Age,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,...,Payment_of_Min_Amount_NM,Payment_of_Min_Amount_No,Payment_of_Min_Amount_Yes,Payment_Behaviour_!@9#%8,Payment_Behaviour_High_spent_Large_value_payments,Payment_Behaviour_High_spent_Medium_value_payments,Payment_Behaviour_High_spent_Small_value_payments,Payment_Behaviour_Low_spent_Large_value_payments,Payment_Behaviour_Low_spent_Medium_value_payments,Payment_Behaviour_Low_spent_Small_value_payments
0,25.0,62214.360,4997.530000,9.0,7.0,19.0,9.0,58.0,20.0,1.05,...,False,False,True,False,False,False,True,False,False,False
1,33.0,14613.650,1334.804167,8.0,NaN,22.0,6.0,20.0,16.0,9.63,...,False,False,True,False,False,False,True,False,False,False
2,56.0,101112.600,8600.050000,8.0,6.0,4.0,2.0,18.0,20.0,1.42,...,False,True,False,False,True,False,False,False,False,False
3,50.0,140390.320,11888.193333,5.0,2.0,4.0,3.0,3.0,0.0,4.67,...,False,True,False,False,False,True,False,False,False,False
4,24.0,20188.020,1851.335000,4.0,3.0,6.0,3.0,10.0,5.0,1.31,...,False,True,False,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44202,40.0,15366.215,996.517917,4.0,4.0,15.0,1.0,24.0,13.0,8.58,...,False,True,False,False,False,False,False,False,False,True
44203,19.0,72125.200,6094.433333,3.0,5.0,12.0,2.0,7.0,19.0,9.52,...,False,False,True,False,True,False,False,False,False,False
44204,32.0,20723.800,1611.983333,7.0,9.0,23.0,6.0,60.0,26.0,18.00,...,True,False,False,False,False,False,False,False,False,True
44205,18.0,16532.330,1416.694167,5.0,6.0,18.0,7.0,25.0,12.0,22.01,...,False,False,True,False,False,False,False,True,False,False


### Train and predict

In [35]:
rf_clf = RandomForestClassifier(random_state=42)
rf_clf.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [36]:
preds = rf_clf.predict(X_test)
preds

array([ 0,  0,  0, ...,  0, -1,  0], shape=(8841,))

In [37]:
subtask5_rows = []
for id_, pred in zip(df_test["ID"], preds):
    subtask5_rows.append((5, id_, pred))

## Save answers

In [38]:
submission_rows = subtask1_rows + subtask2_rows + subtask3_rows + subtask4_rows + subtask5_rows
df_submission = pd.DataFrame(submission_rows, columns=["subtaskID", "datapointID", "answer"])
df_submission.to_csv("submission.csv", index=False)

## Submission results

Subtask 1:
- Accuracy: 1
- Score: 4/4

Subtask 2:
- Accuracy: 1
- Score: 5/5

Subtask 3:
- Accuracy: 1
- Score: 5/5

Subtask 4:
- Accuracy: 1
- Score: 6/6

Subtask 5:
- Accuracy: 0.79
- Score: 80/80